In [21]:
# module import
import numpy as np
import pickle
import re
from gensim import corpora
from gensim.models.ldamodel import LdaModel as LDA
import os
import random

In [7]:
# 경로 설정
root_path = "/content/drive/My Drive/멀티캠퍼스/[혁신성장] 인공지능 자연어처리 기반/[강의]/조성현 강사님"
data_path = f"{root_path}/dataset"

# _1_. 확률분포 생성 후 데이터 만들기

확률 분포이므로 둘 다 더했을 때 1이 되어야 한다.
* `theta`: 토픽 대 문서 분포.
* `beta`: 단어 대 토픽 분포.


In [3]:
# 확률 분포 array 생성
theta = np.array([[0.05, 0.15, 0.50, 0.20, 0.10],
                  [0.05, 0.05, 0.20, 0.10, 0.60],
                  [0.60, 0.15, 0.05, 0.10, 0.10]])

beta = np.array([[0.1, 0.1, 0.2, 0.2, 0.2, 0.1, 0.05, 0.05],
                 [0.15, 0.25, 0.2, 0.05, 0.05, 0.15, 0.1, 0.05],
                 [0.1, 0.2, 0.25, 0.05, 0.25, 0.05, 0.05, 0.05],
                 [0.2, 0.2, 0.05, 0.05, 0.15, 0.15, 0.15, 0.05],
                 [0.1, 0.2, 0.25, 0.1, 0.1, 0.05, 0.15, 0.05]])

* 단어 리스트 생성: 각각의 단어는 `beta[0]`, `beta[1]`, ..., `beta[4]`의 확률 분포를 따른다.

In [4]:
w1 = ["분기", "매출", "영업이익", "영업외이익", "실적", "증가", "감소", "계약"] # beta[0] 확률분포 따름
w2 = ["수주", "계약", "발주", "호재", "기대", "매출", "실적", "약세"] # beta[1] 확률분포 따름
w3 = ["주가", "약세", "하락", "부정적", "약화", "예상", "감소", "영업이익"]
w4 = ["원화", "환율", "영업이익", "매출", "강세", "약세", "달러", "발행"]
w5 = ["자본", "신주", "발행", "전환", "보통주", "기준일", "배정", "발주"]
words = [w1, w2, w3, w4, w5]

* `sentences` : 만들어질 수 있는 문장. 단어 조합으로부터 뽑아 낸다.
* `topic` : 만들 수 있는 주제 조합.

In [5]:
sentences = []
topic = []

for i in range(500):
    # theta 1개 랜덤 선택
    t = np.random.choice([0, 1, 2]) # theta가 0, 1, 2 인덱스 중 하나

    # 단어 50개짜리 문장 생성
    doc = []
    for j in range(50):
        # 다항 분포로 토픽 1개 선택
        x = np.random.multinomial(1, theta[t]) # theta[t] 다항분포로부터 1개 선택
        z = np.argmax(x) # 토픽 번호

        # z번째 토픽에서 1개 단어 선택
        x_word = np.random.multinomial(1, beta[z])
        w = np.argmax(x_word) # 단어 인덱스 번호
        
        # 단어 선택
        word_chosen = words[z][w] # z번째 토픽에서 w번째 단어
        print(word_chosen)
        doc.append(word_chosen)        
        
    sentences.append(doc)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
신주
신주
원화
발행
전환
발행
발행
배정
주가
배정
보통주
영업이익
발행
발주
주가
자본
신주
감소
배정
신주
매출
약화
영업외이익
발행
발행
약화
발행
발행
분기
발행
보통주
자본
약화
자본
기준일
신주
영업이익
배정
전환
발행
보통주
영업외이익
전환
달러
신주
약세
달러
전환
발행
영업이익
계약
계약
매출
실적
수주
배정
매출
영업이익
약세
배정
실적
영업외이익
실적
수주
약세
매출
계약
영업외이익
영업이익
계약
영업외이익
배정
영업이익
매출
영업외이익
영업이익
계약
매출
영업이익
실적
매출
영업이익
영업이익
자본
하락
배정
실적
영업외이익
발주
분기
분기
배정
영업외이익
증가
원화
매출
약세
실적
실적
실적
매출
호재
매출
분기
환율
계약
자본
원화
영업외이익
실적
영업이익
분기
증가
영업이익
영업이익
신주
영업외이익
달러
영업이익
분기
보통주
매출
매출
영업이익
증가
하락
발행
매출
실적
실적
매출
발행
영업이익
증가
영업외이익
약화
발주
분기
분기
증가
발주
매출
영업외이익
약세
실적
매출
발행
발행
영업이익
약화
증가
원화
감소
실적
환율
약화
영업이익
영업외이익
실적
영업이익
달러
발주
영업외이익
감소
신주
실적
약세
매출
실적
분기
자본
실적
실적
신주
감소
실적
계약
발행
달러
영업이익
매출
영업이익
분기
원화
계약
실적
영업이익
약세
영업이익
영업외이익
감소
환율
영업이익
전환
약세
매출
실적
영업외이익
배정
감소
영업외이익
영업이익
하락
영업외이익
실적
영업이익
매출
주가
영업외이익
영업이익
영업외이익
증가
수주
분기
실적
약세
주가
계약
하락
실적
계약
매출
강세
달러
실적
증가
매출
수주
영업외이익
약세
감소
영업이익
발행
실적
실적
증가
영업외이익
약세
계약
영업이익
매출
영업이익
영업외이익
실적
약세
약화
약화
계약
영업이익
하락
약세
약세
달러
자본
자본
원화
약화
강세
하락
약화
약세
호재
계약
기준일
보통주
약화
약세
약세
약화
원화
약화
약세
실적
자본
발행
영

* 데이터 확인 및 저장

In [13]:
print(sentences)

# 데이터 저장
filename = f"{data_path}/8-6-1.genDoc.pickle"
if not os.path.exists(filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)

with open(filename, 'wb') as f:
    pickle.dump(sentences, f, pickle.HIGHEST_PROTOCOL)

[['예상', '강세', '하락', '강세', '계약', '배정', '발행', '달러', '예상', '환율', '실적', '배정', '발행', '하락', '매출', '발주', '원화', '부정적', '주가', '감소', '감소', '계약', '계약', '발행', '약화', '하락', '영업이익', '기준일', '강세', '보통주', '약화', '주가', '강세', '감소', '수주', '주가', '주가', '약세', '주가', '발주', '주가', '감소', '하락', '약세', '하락', '감소', '예상', '하락', '계약', '환율'], ['달러', '발행', '신주', '환율', '주가', '계약', '실적', '감소', '주가', '영업이익', '약세', '약화', '달러', '매출', '약세', '부정적', '약세', '약화', '하락', '약세', '주가', '약세', '실적', '하락', '영업이익', '영업이익', '영업이익', '부정적', '하락', '신주', '하락', '발행', '약화', '보통주', '분기', '실적', '주가', '예상', '약화', '부정적', '하락', '기준일', '약화', '부정적', '약화', '달러', '하락', '신주', '달러', '계약'], ['계약', '수주', '영업이익', '매출', '실적', '기준일', '실적', '실적', '영업이익', '영업외이익', '영업이익', '매출', '계약', '신주', '영업외이익', '실적', '계약', '계약', '영업외이익', '발주', '영업이익', '실적', '계약', '분기', '분기', '실적', '계약', '매출', '발주', '실적', '영업외이익', '보통주', '예상', '수주', '강세', '원화', '매출', '원화', '실적', '분기', '약세', '매출', '발행', '신주', '영업이익', '영업이익', '영업이익', '주가', '매출', '수주'], ['보통주', '약화', '영업외이익', '계약', '강세', '수주', '영업이익

IsADirectoryError: ignored

# _2_. LDA

 `sentences`를 가지고 LDA 수행.

In [15]:
# doc2bow 생성
vocab = corpora.Dictionary(sentences)
bow = [vocab.doc2bow(s) for s in sentences]

print(vocab)
print(bow)

Dictionary(29 unique tokens: ['감소', '강세', '계약', '기준일', '달러']...)
[[(0, 5), (1, 4), (2, 4), (3, 1), (4, 1), (5, 1), (6, 2), (7, 3), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 2), (15, 1), (16, 3), (17, 1), (18, 6), (19, 6), (20, 2)], [(0, 1), (2, 2), (3, 1), (4, 4), (5, 1), (7, 2), (9, 1), (10, 4), (12, 3), (13, 5), (14, 6), (15, 4), (16, 1), (18, 4), (19, 6), (20, 1), (21, 1), (22, 3)], [(1, 1), (2, 6), (3, 1), (5, 6), (6, 2), (7, 1), (9, 1), (11, 3), (12, 8), (13, 1), (15, 7), (16, 1), (17, 2), (18, 1), (21, 3), (22, 2), (23, 4)], [(0, 2), (1, 1), (2, 3), (5, 2), (6, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 5), (13, 3), (14, 2), (15, 10), (17, 1), (21, 4), (23, 6), (24, 5)], [(1, 1), (2, 1), (3, 2), (4, 2), (5, 3), (6, 1), (7, 4), (8, 4), (9, 3), (12, 1), (13, 3), (14, 2), (15, 1), (16, 1), (18, 1), (19, 3), (22, 7), (24, 1), (25, 4), (26, 5)], [(0, 2), (2, 1), (4, 1), (5, 1), (7, 4), (8, 1), (10, 3), (12, 1), (13, 9), (14, 9), (15, 2), (18, 3), (19, 5), (20, 5), (23, 

In [16]:
# LDA 진행
model = LDA(bow,
            num_topics=5,
            id2word=vocab)

In [23]:
# 문서별 topic 번호 확인
doc_topics = model.get_document_topics(bow) # 500개 문서 각각에 대한 토픽
sample_nums = random.sample(list(range(0, 300)), 20) # 20개 문서 랜덤 추출

for num in sample_nums:
    topics = np.array(doc_topics[num])
    most_likely_topic = int(topics[np.argmax(topics[:, 1]), 0])
    print(f"{num}번째 문서: {sentences[num]}")
    print(f"   주제: {most_likely_topic}") # 여기는 theta 인덱스
    print("")

51번째 문서: ['수주', '실적', '계약', '발주', '실적', '매출', '매출', '실적', '영업외이익', '증가', '영업이익', '계약', '영업이익', '분기', '실적', '증가', '영업외이익', '영업외이익', '신주', '전환', '하락', '발주', '환율', '계약', '실적', '증가', '발행', '영업이익', '강세', '발행', '영업이익', '배정', '배정', '실적', '발행', '배정', '실적', '매출', '매출', '실적', '영업외이익', '영업외이익', '분기', '달러', '증가', '약세', '약화', '영업외이익', '실적', '발행']
   주제: 3

132번째 문서: ['부정적', '약세', '하락', '약세', '약화', '원화', '수주', '약화', '약세', '발행', '부정적', '약세', '감소', '배정', '약화', '계약', '약세', '하락', '하락', '약화', '약화', '계약', '주가', '약세', '배정', '환율', '약화', '발주', '약세', '약세', '약화', '하락', '배정', '하락', '하락', '신주', '매출', '약화', '하락', '약세', '하락', '원화', '약화', '하락', '하락', '강세', '수주', '영업외이익', '약세', '약세']
   주제: 0

275번째 문서: ['매출', '약세', '약화', '예상', '하락', '원화', '영업이익', '약화', '원화', '약화', '원화', '약세', '기준일', '하락', '하락', '매출', '약세', '자본', '발행', '약화', '하락', '분기', '강세', '감소', '원화', '부정적', '매출', '발행', '약화', '영업이익', '영업외이익', '환율', '강세', '발주', '발행', '원화', '영업이익', '약세', '발행', '분기', '환율', '약세', '약화', '발행', '호재', '수주', '기대', '원화', '약세', '약세']
   주제:

In [24]:
# topic-term 행렬에서 중요 단어 표시
for i in range(5): # 주제 5개밖에 없다!
    topic_term = model.get_topic_terms(i, topn=10)
    indices = [idx for idx, _ in topic_term]
    print("topic-{:2d} : ".format(i+1))
    for idx in indices:
        print("       ", "{:s}".format(vocab[idx]))
    print("")

topic- 1 : 
        약세
        발행
        약화
        하락
        신주
        영업이익
        배정
        발주
        원화
        보통주

topic- 2 : 
        실적
        영업이익
        약세
        발행
        약화
        영업외이익
        매출
        계약
        하락
        신주

topic- 3 : 
        발행
        신주
        하락
        배정
        자본
        약세
        전환
        보통주
        발주
        약화

topic- 4 : 
        실적
        영업이익
        영업외이익
        계약
        매출
        분기
        감소
        발주
        증가
        약세

topic- 5 : 
        영업외이익
        매출
        실적
        분기
        영업이익
        계약
        하락
        증가
        발주
        약세



In [20]:
print(len(list(doc_topics)))
list(doc_topics)

500


[[(0, 0.9838873)],
 [(0, 0.83086765), (1, 0.15704459)],
 [(3, 0.98383117)],
 [(3, 0.98388547)],
 [(2, 0.9838372)],
 [(0, 0.9838649)],
 [(0, 0.20688044), (2, 0.7810631)],
 [(1, 0.1809589), (3, 0.73859006), (4, 0.07237322)],
 [(0, 0.24016015), (2, 0.7477457)],
 [(3, 0.46034023), (4, 0.5275359)],
 [(2, 0.98364913)],
 [(1, 0.31197602), (3, 0.6759079)],
 [(3, 0.73772246), (4, 0.25025788)],
 [(2, 0.98387206)],
 [(2, 0.9838867)],
 [(3, 0.6251502), (4, 0.36283284)],
 [(3, 0.98385817)],
 [(0, 0.9838741)],
 [(0, 0.983878)],
 [(0, 0.83247954), (4, 0.15544285)],
 [(2, 0.17192878), (3, 0.8159757)],
 [(2, 0.9839096)],
 [(2, 0.25022468), (3, 0.73769253)],
 [(2, 0.10396716), (3, 0.88394165)],
 [(0, 0.8298882), (1, 0.05997982), (4, 0.10207004)],
 [(0, 0.17373274), (1, 0.20567852), (2, 0.6125029)],
 [(0, 0.7920141), (1, 0.18822539), (4, 0.011714442)],
 [(3, 0.53501517), (4, 0.4528325)],
 [(2, 0.98385334)],
 [(2, 0.9838333)],
 [(0, 0.33029583), (2, 0.6575188)],
 [(0, 0.98387253)],
 [(1, 0.2704245), (3, 0